##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Video prompting with REST

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/rest/Video_REST.ipynb"><img src="../../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


This notebook provides quick code examples that show you how  to prompt the Gemini API using a video file with `curl`. In this case, you'll use a short clip of [Big Buck Bunny](https://peach.blender.org/about/).

You can run this in Google Colab, or you can copy/paste the `curl` commands into your terminal.

## Set up the environment

To run this notebook, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you are running in a different environment, you can store your key in an environment variable. See [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) to learn more.

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to the [API console support center](https://support.google.com/googleapi/answer/6310037).

In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

Install `jq` to help with processing of JSON API responses.

In [ ]:
!apt install -q jq

## Use a video file with the Gemini API

The Gemini API accepts video file formats through the File API. The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API. For this example, you will use the short film "Big Buck Bunny".

> "Big Buck Bunny" is (C) Copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

Note: In Colab, you can also [upload your own files](https://github.com/google-gemini/cookbook/blob/main/examples/Upload_files_to_Colab.ipynb) to use.

In [ ]:
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

With the video file now available locally, generate some metadata that you will use in subsequent steps.

In [8]:
%%bash

VIDEO_PATH="./BigBuckBunny_320x180.mp4"
DISPLAY_NAME="Big Buck Bunny"

# Auto-detect the metadata needed when you upload the video.
MIME_TYPE=$(file -b --mime-type "${VIDEO_PATH}")
NUM_BYTES=$(wc -c < "${VIDEO_PATH}")

echo $VIDEO_PATH $MIME_TYPE $NUM_BYTES

# Colab doesn't allow sharing shell variables between cells, so save them.
cat >./vars.sh <<-EOF
  export BASE_URL="https://generativelanguage.googleapis.com"
  export DISPLAY_NAME="${DISPLAY_NAME}"
  export VIDEO_PATH=${VIDEO_PATH}
  export MIME_TYPE=${MIME_TYPE}
  export NUM_BYTES=${NUM_BYTES}
EOF

./BigBuckBunny_320x180.mp4 video/mp4 64657027


### Start the upload task

Media uploads in the File API are resumable, so the first step is to define an upload task. This initial request gives you a reference you can use for subsequent upload operations, and allows you to query the status of the upload before sending data, in case of network issues during the data transfer.

The API returns the upload URL in the `x-goog-upload-url` header, so take note of that in the response headers - you will send the payload data to this URL.

No payload data (video bytes) are sent in this initial request.

In [15]:
%%bash
. vars.sh

# Create the "new upload" request by providing the relevant metadata.
curl "${BASE_URL}/upload/v1beta/files?key=${GOOGLE_API_KEY}" \
  -D upload-header.tmp \
  -H "X-Goog-Upload-Protocol: resumable" \
  -H "X-Goog-Upload-Command: start" \
  -H "X-Goog-Upload-Header-Content-Length: ${NUM_BYTES}" \
  -H "X-Goog-Upload-Header-Content-Type: ${MIME_TYPE}" \
  -H "Content-Type: application/json" \
  -d "{'file': {'display_name': '${DISPLAY_NAME}'}}" 2>/dev/null

# Print the status.
head -1 upload-header.tmp

HTTP/2 200 


### Upload video data

Now that you have created the upload task, you can upload the file data by sending bytes to the returned upload URL.

In [16]:
%%bash
. vars.sh

# Extract the upload URL to use from the response headers.
upload_url=$(grep -i "x-goog-upload-url: " upload-header.tmp | cut -d" " -f2 | tr -d "\r")
# The header contains our API key, so don't leave it lying around.
rm upload-header.tmp

# Upload the actual bytes.
curl "${upload_url}" \
  -H "Content-Length: ${NUM_BYTES}" \
  -H "X-Goog-Upload-Offset: 0" \
  -H "X-Goog-Upload-Command: upload, finalize" \
  --data-binary "@${VIDEO_PATH}" >file_info.json 2>/dev/null

# Show the output. You will use it in a later step.
cat file_info.json

{
  "file": {
    "name": "files/4if4o2bqvugf",
    "displayName": "Big Buck Bunny",
    "mimeType": "video/mp4",
    "sizeBytes": "64657027",
    "createTime": "2024-08-26T08:24:56.068012Z",
    "updateTime": "2024-08-26T08:24:56.068012Z",
    "expirationTime": "2024-08-28T08:24:56.049455995Z",
    "sha256Hash": "Zjc4ZjM5NjAzZTY3NzQ5MDdmMmZhYWZhYmYyNmE2NjdmNGE2ZmMzMTc2OWVjMzA0YThhOGY3YzYyZDI4MDUwOA==",
    "uri": "https://generativelanguage.googleapis.com/v1beta/files/4if4o2bqvugf",
    "state": "PROCESSING"
  }
}


### Get file info

After uploading the file, you can verify the API has successfully received the files by querying the [`files.get` endpoint](https://ai.google.dev/api/files#method:-files.get).

`files.get` lets you see the file uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique.

In [19]:
%%bash
. vars.sh

file_uri=$(jq -r ".file.uri" file_info.json)

curl "${file_uri}?key=${GOOGLE_API_KEY}" 2>/dev/null

{
  "name": "files/4if4o2bqvugf",
  "displayName": "Big Buck Bunny",
  "mimeType": "video/mp4",
  "sizeBytes": "64657027",
  "createTime": "2024-08-26T08:24:56.068012Z",
  "updateTime": "2024-08-26T08:25:03.977029Z",
  "expirationTime": "2024-08-28T08:24:56.049455995Z",
  "sha256Hash": "Zjc4ZjM5NjAzZTY3NzQ5MDdmMmZhYWZhYmYyNmE2NjdmNGE2ZmMzMTc2OWVjMzA0YThhOGY3YzYyZDI4MDUwOA==",
  "uri": "https://generativelanguage.googleapis.com/v1beta/files/4if4o2bqvugf",
  "state": "ACTIVE",
  "videoMetadata": {
    "videoDuration": "596s",
    "videoThumbnailBytes": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQH/2wBDAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQH/wAARCAC0AUADASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJ

### Wait for processing

Once the file is uploaded, the file service will perform some pre-processing to prepare the video for use in the LLM. For simple media types this is typically a negligible amount of time, but for video content you may need a short wait.

You can use the `state` field to query if the video is ready for use. If you use it in a prompt prematurely you will see an error like `The File ... is not in an ACTIVE state and usage is not allowed`.

In [ ]:
%%bash

state=$(jq -r ".file.state" file_info.json)
file_uri=$(jq -r ".file.uri" file_info.json)

while [[ "${state}" == "PROCESSING" ]];
do
  echo "Processing video..."
  sleep 5
  # Get the file of interest to check state
  curl "${file_uri}?key=${GOOGLE_API_KEY}" >file_info.json 2>/dev/null
  state=$(jq -r ".state" file_info.json)
done

echo "Video is now ${state}."

Processing video...
Processing video...
Processing video...
Video is now ACTIVE.


### Prompt with the video

Now that the video is uploaded and processed, you can reference it in a prompt.

When assembling your [`contents`](https://ai.google.dev/api/generate-content#request-body), the video can be referenced using a `file_data` part, like this:

```json
{
  "file_data": {
    "mime_type": "video/mp4",
    "file_uri": "https://uri/from/previous/steps"
  }
}
```

Try it yourself with this request.

In [ ]:
%%bash
. vars.sh

file_uri=$(jq ".uri" file_info.json)

model="gemini-1.5-flash"

curl "${BASE_URL}/v1beta/models/${model}:generateContent?key=${GOOGLE_API_KEY}" \
    -H 'Content-Type: application/json' \
    -X POST \
    -d '{
      "contents": [{
        "parts":[
          {"text": "Please describe this file."},
          {"file_data": {
            "mime_type": "'${MIME_TYPE}'",
            "file_uri": '${file_uri}'}}]
        }]
       }' 2>/dev/null >response.json

jq -C .candidates[].content response.json

{
  "parts": [
    {
      "text": "This is a 3D animated short film about a big bunny that gets tired of his usual diet of grass. He wants to try something different like fruit and learns the value of what he has. The film's title is Big Buck Bunny. "
    }
  ],
  "role": "model"
}


## Further reading

The File API lets you upload a variety of multimodal MIME types, including images, audio, and video formats. The File API handles inputs that can be used to generate content with the [content generation endpoint](https://ai.google.dev/api/generate-content).

* Read the [`File API`](https://ai.google.dev/api/files) reference.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length.